In [1]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

In [31]:
from transformers import ViTFeatureExtractor, ViTModel

In [32]:
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

In [2]:
feature_extractor

ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}

In [3]:
import numpy as np

In [26]:
import os

In [ ]:
for f in 

In [ ]:
biggest = 0
for f in 

In [12]:
a = np.load("/mnt/c/Users/aag12/Documents/subset_moodtheme/00/1062500.npy")
b = np.load("/mnt/c/Users/aag12/Documents/subset_moodtheme/00/1052800.npy")

In [9]:
np.unique(a)

array([-90.      , -89.999794, -89.99772 , ...,  16.418941,  16.554367,
        16.653383], dtype=float32)

In [13]:
a.shape

(96, 10571)

In [14]:
b.shape

(96, 6440)

In [15]:
b

array([[-35.978523, -36.871532, -39.021667, ..., -51.462288, -58.39413 ,
        -78.5748  ],
       [-30.365103, -30.160393, -30.31949 , ..., -53.218292, -50.194256,
        -71.49284 ],
       [-25.871809, -21.388107, -20.183218, ..., -51.226227, -45.24926 ,
        -64.30715 ],
       ...,
       [-90.      , -90.      , -90.      , ..., -90.      , -90.      ,
        -90.      ],
       [-90.      , -90.      , -90.      , ..., -90.      , -90.      ,
        -90.      ],
       [-90.      , -90.      , -90.      , ..., -90.      , -90.      ,
        -90.      ]], dtype=float32)

In [22]:
def mul(a):
    e = 1
    for v in a.shape:
        e = e * v
    return e

In [28]:
a.shape

(96, 10571)

In [27]:
(3 * 244 * 244)

178608

In [30]:
(3 * 244 * 244) - mul(b)

-439632

In [25]:
(3 * 244 * 244) - mul(a)

-836208